## Efficient Simulation in the Inventory Model


#### John Stachurski

Simulation can be time consuming, especially if we need to simulate many paths

And it's often true that large sample sizes are necessary to get an accurate picture

Let's look at speeding up our simulations using [Numba](https://numba.pydata.org/)

In [5]:
import numpy as np
from numba import jit, njit, prange

Parameters:

In [6]:
s, S = 10, 100
p = 0.4
initial_x = 50.0

### Simulating One Path

Let's look at simulating one path with minimal optimization.

(There's at least some efficiency gain over pure Python because we'll use a vectorized function to generate the random variables.)

In [7]:
def sim_one_inventory_path(sim_length=10_000_000):
        
    dvals = np.random.geometric(p, size=sim_length-1) - 1
    X = np.empty(sim_length, dtype=np.int64)
    X[0] = initial_x
    
    for t, d in enumerate(dvals):
        x = X[t]
        if x <= s:
            X[t+1] = max(S - d, 0)
        else:
            X[t+1] = max(x - d, 0)
            
    return X

How fast does this run?

In [8]:
%%time
X = sim_one_inventory_path()

CPU times: user 7.36 s, sys: 94 ms, total: 7.45 s
Wall time: 7.56 s


Now let's build a jitted version.

In [9]:
@jit(nopython=True)
def update(x):        
    d = np.random.geometric(p) - 1
    if x <= s:
        y = np.maximum(S - d, 0)
    else:
        y = np.maximum(x - d, 0)
    return y

In [10]:
update(20)

20

In [11]:
@jit(nopython=True)
def sim_one_path_jitted(sim_length=10_000_000):
    X = np.empty(sim_length, dtype=np.int64)
    X[0] = initial_x
    for t in range(sim_length-1):
        X[t+1] = update(X[t])

This is **much** faster:

In [12]:
%%timeit
X = sim_one_path_jitted()

309 ms ± 1.77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Task: Calculate Stationary Order Rate

As we've seen, the inventory model has a stationary distribution.

Let's call it $\psi^*$, with the corresponding random variable denoted $X^*$.

For a large population with independent demand shocks,

$$ \mathbb P\{X^* = k\} \simeq \text{fraction of firms with inventory } k $$

We are interested in calculating

$$ \mathbb P\{X^* \leq s\} \simeq \text{fraction of firms currently placing orders } $$

The law of large numbers for stationary and ergodic sequences tells us that

$$ \mathbb P\{X^* \leq s\} \simeq \frac{1}{n} \sum_{t=1}^n \mathbb 1\{X_t \leq s\} $$

In other words, we can calculate the stationary probability $\mathbb P\{X^* \leq s\}$ by generating a single time series and calculating the fraction of time that it spends below $s$.

We can do this fast with the jitted code above, thanks to Numba.

#### Exploiting parallelization

However, the above approach is difficult to parallelize because it's inherently sequential

An approach that's easier to parallelize is to 

* pick some large value $T$

* generate many independent observations $X^i_T$ of $X_T$

* calculate the fraction of observations that are less than $s$

This works because 

$$ \mathbb P\{X^* \leq s\} \simeq \mathbb P \{X_T \leq s\} $$

when $T$ is large (since, as previously discussed, $\psi_t \to \psi^*$).

Thus, letting $m$ be the number of paths we'll generate, our goal is to fix large $T$ and compute

$$ p(m, T) = \frac{1}{m} \sum_{i=1}^m \mathbb 1\{X_T^i \leq s\} $$

### Simulating Many Paths

Here's a jitted (and hence fast) piece of code for simulating many paths.

In [16]:
@njit
def sim_prob(m=500_000, T=1000):
    
    is_below_s = 0
    for i in range(m):
        x = initial_x
        for t in range(T):
            x = update(x)
        if x <= s:
            is_below_s += 1
    p_m_T = is_below_s / m
    return p_m_T

Let's get a reading on the time.

In [22]:
%%time
p = sim_prob()

CPU times: user 15.9 s, sys: 128 ms, total: 16 s
Wall time: 16.1 s


In [23]:
p

0.016104

Now let's try a simple parallelization:

In [24]:
@njit(parallel=True)
def sim_prob_parallel(m=500_000, T=1000):
    
    is_below_s = 0
    for i in prange(m):
        x = initial_x
        for t in range(T):
            x = update(x)
        if x <= s:
            is_below_s += 1
    p_m_T = is_below_s / m
    return p_m_T


In [26]:
%%time
p = sim_prob_parallel()

CPU times: user 21.2 s, sys: 48.8 ms, total: 21.3 s
Wall time: 5.62 s


In [27]:
p

0.016468

# Homework

Recall our inventory problem where inventory follows


\begin{equation*}
  X_{t+1} = 
  \begin{cases}
      ( S - D_{t+1})^+ & \quad \text{if } X_t \leq s \\
      ( X_t - D_{t+1} )^+ &  \quad \text{if } X_t > s
  \end{cases}
\end{equation*}

The demand sequence is IID and obeys

$$ \mathbb P \{D_t = d\} = (1 - p)^d p $$

for $d = 0, 1, \ldots$.  Here $p$ is a parameter in $(0, 1)$.

Let

$$ 
    h_s(x) = x + (S - x) \mathbb 1\{x \leq s \}
$$

denote stock on hand after the restocking decision is made.  Profits are given by

$$ 
    \pi(X_t, D_{t+1}) 
    = \min\{h_s(X_t), D_{t+1}\} - c \mathbb 1\{X_t \leq s\}
$$

Here $c$ is a fixed cost of restocking.  We are assuming that the marginal cost of stock is zero for simplicity, and that the unit price of output is 1.

Maximize long run average profits for the firm by choosing $s$ optimally.  You can do this visually, if you like, by plotting profits as a function of $s$.

In doing so, assume that $c=0.1$, $S=20$ and $p=0.5$.

Use simulation and the law of large numbers.  Try to optimize the execution speed of your implementation.